In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

from bokeh.io import push_notebook, show, output_notebook
import bokeh.layouts as bl
import bokeh.plotting as bp
import bokeh.models as bm
output_notebook()

In [ ]:
# The following code is adopted from Pat's Rolling Rain N-Year Threshold.pynb
# Loading in hourly rain data from CSV, parsing the timestamp, and adding it as an index so it's more useful

rain_df = pd.read_csv('data/ohare_full_precip_hourly.csv')
rain_df['datetime'] = pd.to_datetime(rain_df['datetime'])
rain_df = rain_df.set_index(pd.DatetimeIndex(rain_df['datetime']))
rain_df = rain_df['19700101':]
chi_rain_series = rain_df['HOURLYPrecip'].resample('1H').max()
print(rain_df.shape)

In [ ]:
n_year_threshes = pd.read_csv('data/n_year_definitions.csv')
n_year_threshes = n_year_threshes.set_index('Duration')
n_year_threshes

In [ ]:
n_year_threshes.loc['5-min', '1-year']

In [ ]:
dur_str_to_hours = {
    '5-min':5/60.0,
    '10-min':10/60.0,
    '15-min':15/60.0,
    '30-min':0.5,
    '1-hr':1.0,
    '2-hr':2.0,
    '3-hr':3.0,
    '6-hr':6.0,
    '12-hr':12.0,
    '18-hr':18.0,
    '24-hr':24.0,
    '48-hr':48.0,
    '72-hr':72.0,
    '5-day':5*24.0,
    '10-day':10*24.0
}

def plot_thresh(duration_str, n_years):
    '''
    TODO
    '''
    global rain_df
    global n_year_threshes
    
    thresh = n_year_threshes.loc[duration_str, str(n_years) + '-year']
    
    duration = dur_str_to_hours[duration_str]
    duration = max(duration, 1) # cannot upsample to more frequent than hourly
    # TODO: want to throw warning?
    
    rain_line = chi_rain_series.rolling(window=int(duration), min_periods=0).sum()
    
    # Create plot
    p = bp.figure(plot_width=800, plot_height=600,
                  x_axis_type="datetime",
                  tools="xwheel_zoom,xpan,hover,save,reset")
    
    p.line(rain_line.index, rain_line.values,
           legend='moving cumulative rain')
    
    x_limits = [rain_line.index.min(), rain_line.index.max()]
    p.line(x_limits, [thresh, thresh],
           color="firebrick", legend=str(n_years) + '-year ' + duration_str + ' threshold')
    
    p.legend.location = "top_left"
    
    hover = p.select(dict(type=bm.HoverTool))
    hover.tooltips = [
        ("rain (in)", "$y"),
        ]
    
    p.toolbar.active_drag = None
    
    bp.show(p)
    
    return p

In [ ]:
plot_thresh('48-hr', 100)